In [1]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import autoFill as af
from ipywidgets import VBox
import requests
from PIL import Image as img
#from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from io import BytesIO
from IPython.display import display
from ipywidgets import AppLayout
import pickle
# need ipympl


In [2]:
df_poster = pd.read_csv('poster.csv')

In [3]:
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [4]:
df_ratings = df_ratings[df_ratings["userId"] < 100]

In [5]:
df_links = pd.read_csv('ml-latest-small/links.csv')

In [6]:
df_movies = pd.read_csv('ml-latest-small/movies.csv')

In [7]:
def get_poster_by_id(movie_id):
    imdb_id = df_links[df_links["movieId"] == movie_id].iloc[0]["imdbId"]
    return df_poster[df_poster["imdbId"] == imdb_id].iloc[0]["Poster"]

def title_to_id(title):
    return df_movies[df_movies["title"] == title].iloc[0]["movieId"]

def id_to_title(movie_id):
    return df_movies[df_movies["movieId"] == movie_id].iloc[0]["title"]

def get_list_of_recommendation(mov):
    fav_movie_id = title_to_id(mov)
    fav_movie_index = movies_list.index(fav_movie_id)
    corr_fav_movie = corr_matrix[fav_movie_index]
    top_10_idx = np.argsort(corr_fav_movie)[-11:-1]
    results = []
    for idx in top_10_idx[::-1]:
        results.append(id_to_title(movies_list[idx]))
    return results

rating_utility_matrix = df_ratings.pivot_table(values = 'rating', index = 'userId', columns = 'movieId', fill_value = 0)
X = rating_utility_matrix.T


SVD = TruncatedSVD(n_components=30)
transposed_matrix = SVD.fit_transform(X)


'''
file = open('transposed_matrix', 'wb')
pickle.dump(transposed_matrix, file)
file.close()

file = open('transposed_matrix', 'rb')
transposed_matrix = pickle.load(file)
file.close()
'''
corr_matrix = np.corrcoef(transposed_matrix)

movies = rating_utility_matrix.columns
movies_list =  list(movies)

In [8]:

plt.ioff()
fig, axs = plt.subplots(2,5,figsize=(15, 8))
fig.suptitle('10 Movie Recommendation')
#fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
lineimage = []
for i in range(2):
    for j in range(5):
        lineimage.append(None)

for i in range(2):
    for j in range(5):
        axs[i,j].axis("off")
        r = requests.get(get_poster_by_id(94))
        im = img.open(BytesIO(r.content))
        for p in range(im.width):
            for q in range(im.height):
                im.putpixel((p,q),(0,0,0)) 

        lineimage[5*i+j] = axs[i,j].imshow(im)

def update(value):
    #plt.cla()
    results = get_list_of_recommendation(value.new)
    for i in range(2):
        for j in range(5):
            axs[i,j].set_title(results[5*i+j],fontsize=8)
            try:
                r = requests.get(get_poster_by_id(title_to_id(results[5*i+j])))
                im = img.open(BytesIO(r.content))
                lineimage[5*i+j].set_data(im)
            except:
                r = requests.get(get_poster_by_id(94))
                im = img.open(BytesIO(r.content))
                for p in range(im.width):
                    for q in range(im.height):
                        im.putpixel((p,q),(0,0,0)) 

                draw = ImageDraw.Draw(im)
                font = ImageFont.truetype("ABeeZee-Regular.otf", 24)
                title = results[5*i+j].split(' ')
                for k in range(len(title)):
                    draw.text((5, 5+k*24),title[k],(255,255,255),font=font)
                lineimage[5*i+j].set_data(im)
            display(5*i+j)
    fig.canvas.draw()
    fig.canvas.flush_events()

options = [] 
for movie in movies_list:
    options.append(id_to_title(movie)) #df_movies["title"].to_list()
autofill = af.autoFill(options,callback=update)
app=AppLayout(
    center=fig.canvas
)
app

AppLayout(children=(Canvas(footer_visible=False, header_visible=False, layout=Layout(grid_area='center'), tool…

In [9]:
display(VBox([autofill]))

0

1

2

3

4

5

6

7

8

9